### Importing required pakages 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
import os
os.getcwd()

'D:\\Abhishek\\Personal\\Springboard\\Springboard_DSC\\Capstone Projects'

### Importing Data from the CSV file
Missing values in the file are represented by '?'. 

In [3]:
%%time
#load original data into dataframe
file ='D:\\Abhishek\\Personal\\Springboard\\dataset_diabetes\\diabetic_data.csv'
df1 = pd.read_csv(file, na_values = '?')

Wall time: 694 ms


<string>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.


### Understanding the Data. 
Number of columns, whether it was imported correctly, number of missing values etc.

In [4]:
#check shape of the dataset and its columns
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        99493 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      3197 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  61510 non-null object
medical_specialty           51817 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101766 non

(101766, 50)

In [5]:
#Checking if the data was imported correctly
df1.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
len(df1['weight'].unique())
df1.race.mode()[0]

'Caucasian'

Converting the objects to categories. This would help save memory space.

## Missing Value Analysis

### Finding our features with missing value

In [7]:
# Creating a empty table to store Missing Values Count
m_table = pd.DataFrame(columns = ['feature_name', 'null_values', '%missing', 'category_count'])

#Counting the missing values and appending it to m_table
for col in df1.columns:
    null_value = len(df1) - df1[col].count()
    missing = round((null_value/len(df1))*100,2)
    count = len(df1[col].unique())
    m_table = m_table.append(pd.Series([col, null_value, missing,count],index = m_table.columns),ignore_index=True)

#Displaying features which has missing values in descending order
missing = m_table[m_table.null_values != 0].sort_values(by = '%missing',ascending = False).reset_index(drop = True)
display('Missing Value Table')
display(missing)
print('\n')

#Displaying features having only single category
c_count = m_table[m_table.category_count == 1].sort_values(by = '%missing',ascending = False).reset_index(drop = True)
display('Single Category Features')
display(c_count)

'Missing Value Table'

,feature_name,null_values,%missing,category_count
0,weight,98569,96.86,10
1,medical_specialty,49949,49.08,73
2,payer_code,40256,39.56,18
3,race,2273,2.23,6
4,diag_3,1423,1.40,790
5,diag_2,358,0.35,749
6,diag_1,21,0.02,717


'Single Category Features'

,feature_name,null_values,%missing,category_count
0,examide,0,0.0,1
1,citoglipton,0,0.0,1


#### 6 features have missing values.
1) Weight have ~97% missing values. There is poor interpretability of the missing values, so it is best to drop this column. 

2) medical_speciality and payer_code have 40-50% missing values. I have decided to drop it but there are ways to deal with it. I will experiment and use this columns to see if it increases the accuracy of the model. Method to include these columns would be to create a separate category of 'missing'. 

3) race, diag_3, diag_2, diag_1  have only <=2% missing values, so missing rows can be dropped or be replace with mode of the column values. 

5) When all diag are missing, then also it is a bad data. So secondly, removing all rows where where all 3 diag are missing

6) A missing first diagnosis while data have diagnosis second and third, is also a bad data. But second and third diagnosis can give us some idea about the readmission. So we  can either keep this missing values under "missing" category or just ignore the rows. I am going with first one.  

7) For the remaining missing values of Race, it is repalced by mode i.e. Caucasian. 

8) In gender, there are values like "Unknown/Invalid" which are missing. These columns are also dropped




In [8]:
# Droping weight, medical_speciality and payer_code
drop_col1 = ['weight', 'medical_specialty','payer_code']
df_clean =df1.drop(drop_col1,axis = 1)

# Droping rows with missing values in all 4 columns of race, diag_3, diag_2 and diag_1
drop_col2 = ['diag_1','diag_2', 'diag_3']
df_clean = df_clean.dropna(subset = drop_col2, how = 'all')

# Replacing missing values of 'diag_1, diag_2, diag_3' with Missing = 'M'
values = {'diag_1': '0', 'diag_2': '0', 'diag_3': '0'}
df_clean = df_clean.fillna(value = values)

# Droping rows with gender value  as 'Unknown/Invalid'
df_clean = df_clean[df_clean.gender != 'Unknown/Invalid']

# Replacing missing values of 'race' with mode 
df_clean = df_clean.fillna(value={'race': df1.race.mode()[0]})

#df_clean.info()
df_clean.shape

(101762, 47)

Following code will see if there are any more missing values or not

In [9]:
# Creating a empty table to store Missing Values Count
c_table = pd.DataFrame(columns = ['feature_name', 'null_values', '%missing', 'category_count'])

#Counting the missing values and appending it to m_table
for col in df_clean.columns:
    null_value = len(df_clean) - df_clean[col].count()
    missing = round((null_value/len(df_clean))*100,2)
    count = len(df_clean[col].unique())
    c_table = c_table.append(pd.Series([col, null_value, missing,count],index = c_table.columns),ignore_index=True)

#Displaying features which has missing values in descending order
display('Missing Value Table')
display(c_table)
print('\n')

'Missing Value Table'

,feature_name,null_values,%missing,category_count
0,encounter_id,0,0.0,101762
1,patient_nbr,0,0.0,71514
2,race,0,0.0,5
3,gender,0,0.0,2
4,age,0,0.0,10
5,admission_type_id,0,0.0,8
6,discharge_disposition_id,0,0.0,26
7,admission_source_id,0,0.0,17
8,time_in_hospital,0,0.0,14
9,num_lab_procedures,0,0.0,118


## Further Cleaning the Data

Since we are trying to predict readmissions, those patients who died during this hospital admission, have zero probability of readmission. So we should remove those records (discharge_disposition = 11, 19,20,21).

In [10]:
# List of Dicharge disposition ids saying that patient got expired or met with a death 
expired = [11,19,20,21]
df_clean = df_clean[~df_clean.discharge_disposition_id.isin(expired)]

While doing missing value analysis, I found that there are two columns where all records have same values (examide and citoglipton). So it would be better to drop these columns also since it cannot provide any information about readmission. 

In [11]:
# Dropping examide and citoglipton
no_info = ['examide','citoglipton']
df_clean = df_clean.drop(no_info,axis = 1)
df_clean.shape


(100110, 45)

Diagnosis columns have more than >600 categories. They are ICD9 Codes describing specific diseases. We can club them in larger categories in accordance with ICD9 to reduce the categories. 

In [12]:
diag_cols = ['diag_1','diag_2','diag_3']
for col in diag_cols:
    df_clean[col] = df_clean[col].str.replace('E','-')
    df_clean[col] = df_clean[col].str.replace('V','-')
    condition = df_clean[col].str.contains('250')
    df_clean.loc[condition,col] = '250'



In [13]:
df_clean[diag_cols] = df_clean[diag_cols].replace('',np.nan).astype(float)

# diagnosis grouping
for col in diag_cols:
    df_clean['temp']=np.nan
    
    condition = df_clean[col]==250
    df_clean.loc[condition,'temp']='Diabetes'
    
    condition = (df_clean[col]>=390) & (df_clean[col]<=458) | (df_clean[col]==785)
    df_clean.loc[condition,'temp']='Circulatory'
    
    condition = (df_clean[col]>=460) & (df_clean[col]<=519) | (df_clean[col]==786)
    df_clean.loc[condition,'temp']='Respiratory'
    
    condition = (df_clean[col]>=520) & (df_clean[col]<=579) | (df_clean[col]==787)
    df_clean.loc[condition,'temp']='Digestive'
    
    condition = (df_clean[col]>=580) & (df_clean[col]<=629) | (df_clean[col]==788)
    df_clean.loc[condition,'temp']='Genitourinary'
    
    condition = (df_clean[col]>=800) & (df_clean[col]<=999)
    df_clean.loc[condition,'temp']='Injury'
    
    condition = (df_clean[col]>=710) & (df_clean[col]<=739)
    df_clean.loc[condition,'temp']='Muscoloskeletal'
    
    condition = (df_clean[col]>=140) & (df_clean[col]<=239)
    df_clean.loc[condition,'temp']='Neoplasms'
    
    condition = df_clean[col] == 0
    df_clean.loc[condition,'temp']='Missing'
    df_clean['temp']=df_clean['temp'].fillna('Others')
    df_clean[col]=df_clean['temp']
    df_clean.drop('temp',axis=1,inplace=True)


In [14]:
df_clean.diag_1.value_counts()

Circulatory        29783
Others             17984
Respiratory        14074
Digestive           9380
Diabetes            8693
Injury              6879
Genitourinary       5054
Muscoloskeletal     4944
Neoplasms           3299
Missing               20
Name: diag_1, dtype: int64

Converting to categories to reduce memory space

In [15]:
#cat_names = ['race','gender','age', 'weight','payer_code','medical_specialty']
#other_cat_names = list(df1.columns)
#other_cat_names = other_cat_names[22:]
cat_names = list(df_clean.select_dtypes(include='object').columns)
df_clean[cat_names] = df_clean[cat_names].astype('category')
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100110 entries, 0 to 101765
Data columns (total 45 columns):
encounter_id                100110 non-null int64
patient_nbr                 100110 non-null int64
race                        100110 non-null category
gender                      100110 non-null category
age                         100110 non-null category
admission_type_id           100110 non-null int64
discharge_disposition_id    100110 non-null int64
admission_source_id         100110 non-null int64
time_in_hospital            100110 non-null int64
num_lab_procedures          100110 non-null int64
num_procedures              100110 non-null int64
num_medications             100110 non-null int64
number_outpatient           100110 non-null int64
number_emergency            100110 non-null int64
number_inpatient            100110 non-null int64
diag_1                      100110 non-null category
diag_2                      100110 non-null category
diag_3                  

In [16]:
df_clean.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
